In [ ]:
!pip install selenium
!pip install html5lib
!pip install python-csv

In [ ]:
class Anime:
    def __init__(self):
        self._name = ''
        self._release = ''
        self._imdb = ''
        self._votes = ''
        self._detail = ''
        self._image = ''
    
    def getAnime(self, temp):
        temp.append(self._name)
        temp.append(self._release)
        temp.append(self._imdb)
        temp.append(self._votes)
        temp.append(self._detail)
        temp.append(self._image)
        

    # def setData(self, name, release, imdb, votes, detail, image):
    #     self._name = name
    #     self._release= release
    #     self._imdb = imdb
    #     self._votes = votes
    #     self._detail = detail
    #     self._image = image


In [ ]:
# Thêm thư viện cho project
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
from bs4 import BeautifulSoup
from config_repository import ConfigRepository
import pandas as pd


In [ ]:
url = 'https://www.imdb.com/search/keyword/?keywords=anime'
url = 'https://myanimelist.net/topanime.php'
domain = urlparse(url).netloc
driver = webdriver.Chrome(executable_path='chromedriver.exe')
driver.get(url)
driver.maximize_window()
sleep(3)
page_source = BeautifulSoup(driver.page_source, "html.parser")

In [ ]:
# domain = urlparse('search/keyword/?keywords=anime').netloc

# if domain:
#     print(domain)
# else: print('no domain')

In [ ]:
def getDataInPage(page_source):

    cfs = ConfigRepository()
    pattern = cfs.getPatternByUrl(url)
    list_records = page_source.select(pattern["list"])
    anime = Anime()
    records = []
    temp = []
    for record in list_records:
        anime._name = record.select_one(pattern["title"]).text
        # print(anime._name)
        subdomain = record.select_one(pattern["title"])["href"]
        if urlparse(subdomain).netloc:
            anime._detail = subdomain
        else:
            anime._detail = domain + subdomain
        # print(anime._detail)
        anime._imdb = record.select_one(pattern["imdb"]).text.split()
    # print(anime._imdb)
        votes = record.select_one(pattern["votes"]).text.split()
        if len(votes) > 1:
            anime._votes = votes[pattern["index-votes"]]
        else:
            anime._votes = votes
    # print(anime._votes)
        release = record.select_one(pattern["release"]).text.split()
    # print(release)
        if len(release) > 2:
            anime._release = (
                release[pattern["index-release-start"]]
                + " - "
                + release[pattern["index-release-end"]]
            )
        else:
            anime._release = record.select_one(pattern["release"]).text
    # print(anime._release)
        anime._image = record.select_one(pattern["thumbnail"])[pattern["get-thumbnail"]]

        anime.getAnime(temp)
        records.append(temp)
    # print(records)
    return records


